1st problem

In [ ]:
import numpy as np
import pandas as pd
import os
import re
from scipy import stats, optimize
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# ==================== 1. ENHANCED DATA PROCESSING ====================

class EnhancedDataProcessor:
    """Enhanced data processor for all 34 seasons"""
    
    def __init__(self, filepath):
        self.filepath = filepath
        self.data = None
        self.season_rules = {}
        self.all_seasons_data = {}
        
    def load_and_clean_data(self):
        """Load and clean data from Excel or CSV"""
        try:
            if self.filepath.endswith(('.xlsx', '.xls')):
                self.data = pd.read_excel(self.filepath)
            else:
                self.data = pd.read_csv(self.filepath)
        except Exception as e:
            print(f"Error loading data: {e}")
            return None
        
        print(f"✓ Original data loaded: {len(self.data)} rows, {len(self.data.columns)} columns")
        
        # Check and handle duplicate data
        original_len = len(self.data)
        
        # Create unique identifier: season + celebrity_name
        self.data['unique_id'] = self.data['season'].astype(str) + '_' + self.data['celebrity_name'].astype(str)
        
        # Find duplicate records
        duplicates = self.data.duplicated(subset=['season', 'celebrity_name'], keep='first')
        if duplicates.any():
            print(f"⚠ Found {duplicates.sum()} duplicate records (same season and celebrity)")
            print("  Removing duplicates...")
            
            # Show duplicate examples
            dup_examples = self.data[duplicates][['season', 'celebrity_name']].head()
            print(f"  Example duplicates:\n{dup_examples}")
            
            # Remove duplicates, keep first record
            self.data = self.data.drop_duplicates(subset=['season', 'celebrity_name'], keep='first')
            print(f"✓ Removed {original_len - len(self.data)} duplicate rows")
        
        print(f"✓ Cleaned data: {len(self.data)} rows")
        print(f"✓ Seasons found: {sorted(self.data['season'].unique())}")
        
        return self.data
    
    def determine_season_rules_enhanced(self):
        """Determine voting rule for each season with enhanced detection"""
        # According to problem description:
        # Seasons 1-2: rank method
        # Seasons 3-27: percentage method  
        # Seasons 28-34: rank method
        for season in self.data['season'].unique():
            if season <= 2:
                self.season_rules[season] = 'rank'
            elif 3 <= season <= 27:
                self.season_rules[season] = 'percentage'
            else:  # season >= 28
                self.season_rules[season] = 'rank'
        
        print(f"\nSeason rules determined for all {len(self.season_rules)} seasons:")
        
        # Print summary
        rank_seasons = [s for s, r in self.season_rules.items() if r == 'rank']
        percentage_seasons = [s for s, r in self.season_rules.items() if r == 'percentage']
        
        print(f"  Rank method seasons: {len(rank_seasons)} (1-2, 28-34)")
        print(f"  Percentage method seasons: {len(percentage_seasons)} (3-27)")
        
        return self.season_rules
    
    def prepare_all_seasons_data(self):
        """Prepare data for all 34 seasons"""
        print("\nPreparing data for all seasons...")
        
        all_seasons = sorted(self.data['season'].unique())
        total_seasons = len(all_seasons)
        
        for idx, season_num in enumerate(all_seasons, 1):
            print(f"  Season {season_num} ({idx}/{total_seasons})...")
            
            season_data = self._prepare_single_season(season_num)
            if season_data is not None:
                self.all_seasons_data[season_num] = season_data
        
        print(f"\n✓ Successfully prepared data for {len(self.all_seasons_data)} seasons")
        return self.all_seasons_data
    
    def _prepare_single_season(self, season_num):
        """Prepare data for a single season"""
        season_data = self.data[self.data['season'] == season_num].copy()
        season_data = season_data.reset_index(drop=True)
        
        if len(season_data) == 0:
            return None
        
        # Determine max weeks
        judge_cols = [col for col in season_data.columns 
                     if 'week' in str(col).lower() and 'judge' in str(col).lower()]
        
        if not judge_cols:
            return None
        
        # Extract week numbers
        week_nums = []
        for col in judge_cols:
            match = re.search(r'week(\d+)_', str(col).lower())
            if match:
                week_nums.append(int(match.group(1)))
        
        if not week_nums:
            return None
            
        max_week = max(week_nums)
        
        # Prepare features
        features = self._extract_features_enhanced(season_data)
        
        # Prepare judge data
        judge_data = self._extract_judge_data_enhanced(season_data, max_week)
        
        # Prepare elimination data
        elimination_data = self._extract_elimination_data_enhanced(season_data, judge_data['active_mask'], max_week)
        
        # Determine rule type
        rule_type = self.season_rules.get(season_num, 'percentage')
        
        # Store contestant names and IDs for later reference
        contestant_info = []
        for idx, row in season_data.iterrows():
            contestant_info.append({
                'index': idx,
                'name': row['celebrity_name'] if 'celebrity_name' in row else f'Contestant_{idx}',
                'original_index': row.name if 'name' in row else idx
            })
        
        return {
            'season_num': season_num,
            'original_data': season_data,
            'features': features,
            'judge_data': judge_data,
            'elimination_data': elimination_data,
            'max_week': max_week,
            'rule_type': rule_type,
            'n_contestants': len(season_data),
            'contestant_info': contestant_info,
            'season_data_indices': season_data.index.tolist()  # Save original indices
        }
    
    def _extract_features_enhanced(self, season_data):
        """Extract enhanced features for fan vote prediction"""
        features_list = []
        feature_names = []
        
        # 1. Age feature (standardized)
        if 'celebrity_age_during_season' in season_data.columns:
            age = season_data['celebrity_age_during_season'].values
            age = np.nan_to_num(age, nan=np.nanmean(age) if not np.all(np.isnan(age)) else 30)
            if np.std(age) > 0:
                age = (age - np.mean(age)) / np.std(age)
            features_list.append(age.reshape(-1, 1))
            feature_names.append('age_std')
        
        # 2. USA flag (binary)
        country_col = None
        for col in ['celebrity_homecountry/region', 'celebrity_homecountry', 'country']:
            if col in season_data.columns:
                country_col = col
                break
        
        if country_col:
            is_usa = season_data[country_col].astype(str).str.contains(
                'United States|USA|U.S.|America', case=False, na=False)
            features_list.append(is_usa.astype(float).values.reshape(-1, 1))
            feature_names.append('is_usa')
        
        # 3. Previous performance (judge scores history)
        judge_score_cols = [col for col in season_data.columns if 'judge' in str(col).lower() and 'score' in str(col).lower()]
        if judge_score_cols:
            judge_scores = season_data[judge_score_cols].values
            mean_judge_scores = np.nanmean(np.where(judge_scores > 0, judge_scores, np.nan), axis=1)
            mean_judge_scores = np.nan_to_num(mean_judge_scores, nan=np.nanmean(mean_judge_scores) if not np.all(np.isnan(mean_judge_scores)) else 7)
            if np.std(mean_judge_scores) > 0:
                mean_judge_scores = (mean_judge_scores - np.mean(mean_judge_scores)) / np.std(mean_judge_scores)
            features_list.append(mean_judge_scores.reshape(-1, 1))
            feature_names.append('avg_judge_score')
        
        # 4. Partner experience
        if 'ballroom_partner' in season_data.columns:
            partner_counts = season_data['ballroom_partner'].value_counts()
            experienced_partners = partner_counts[partner_counts > 2].index.tolist()
            
            for partner in experienced_partners[:3]:
                if pd.notna(partner):
                    partner_feature = (season_data['ballroom_partner'] == partner).astype(float).values
                    features_list.append(partner_feature.reshape(-1, 1))
                    feature_names.append(f'partner_{partner[:10].replace(" ", "_")}')
        
        # 5. Industry popularity
        if 'celebrity_industry' in season_data.columns:
            industry_weights = {
                'Actor': 1.0, 'Singer': 0.9, 'Athlete': 0.8, 'Model': 0.7,
                'Reality TV Star': 0.6, 'Political Figure': 0.5, 'TV Personality': 0.7,
                'Comedian': 0.7, 'Musician': 0.9, 'Dancer': 0.6, 'Actor/Singer': 0.95,
                'Singer/Actor': 0.95, 'Athlete/Model': 0.75
            }
            
            industry_feature = np.zeros(len(season_data))
            for idx, industry in enumerate(season_data['celebrity_industry']):
                if pd.isna(industry):
                    industry_feature[idx] = 0.5
                else:
                    industry_str = str(industry).strip()
                    industry_feature[idx] = industry_weights.get(industry_str, 0.5)
            
            features_list.append(industry_feature.reshape(-1, 1))
            feature_names.append('industry_popularity')
        
        # 6. Add intercept term
        features_list.append(np.ones((len(season_data), 1)))
        feature_names.append('intercept')
        
        # Combine all features
        if features_list:
            X = np.hstack(features_list)
        else:
            X = np.ones((len(season_data), 1))
            feature_names = ['intercept']
        
        return {
            'matrix': X,
            'names': feature_names,
            'n_features': X.shape[1]
        }
    
    def _extract_judge_data_enhanced(self, season_data, max_week):
        """Extract enhanced judge data"""
        n_contestants = len(season_data)
        
        judge_scores = np.zeros((n_contestants, max_week))
        judge_percentages = np.zeros((n_contestants, max_week))
        judge_ranks = np.zeros((n_contestants, max_week))
        active_mask = np.zeros((n_contestants, max_week), dtype=bool)
        
        for week in range(1, max_week + 1):
            # Find judge score columns
            judge_cols = []
            for i in range(1, 5):
                for pattern in [f'week{week}_judge{i}_score', f'Week{week}_Judge{i}_Score']:
                    if pattern in season_data.columns:
                        judge_cols.append(pattern)
                        break
            
            if judge_cols:
                scores = season_data[judge_cols].mean(axis=1).values
                judge_scores[:, week-1] = scores
                active_mask[:, week-1] = scores > 0
                
                # Calculate judge percentages
                active_idx = np.where(active_mask[:, week-1])[0]
                if len(active_idx) > 0:
                    active_scores = scores[active_idx]
                    if np.sum(active_scores) > 0:
                        percentages = active_scores / np.sum(active_scores)
                        judge_percentages[active_idx, week-1] = percentages
            
            # Get judge ranks
            for pattern in [f'Week{week}_Rank', f'week{week}_rank']:
                if pattern in season_data.columns:
                    ranks = season_data[pattern].values
                    ranks = np.nan_to_num(ranks, nan=np.max(ranks[~np.isnan(ranks)]) + 1 
                                          if np.any(~np.isnan(ranks)) else len(season_data))
                    judge_ranks[:, week-1] = ranks
                    break
        
        return {
            'scores': judge_scores,
            'percentages': judge_percentages,
            'ranks': judge_ranks,
            'active_mask': active_mask
        }
    
    def _extract_elimination_data_enhanced(self, season_data, active_mask, max_week):
        """Enhanced elimination data extraction"""
        n_contestants = len(season_data)
        eliminated_idx = [None] * max_week
        
        for week in range(1, max_week):
            current_active = active_mask[:, week-1]
            
            if week < max_week:
                next_active = active_mask[:, week]
                eliminated_candidates = np.where(current_active & ~next_active)[0]
            else:
                eliminated_candidates = np.array([], dtype=int)
            
            if len(eliminated_candidates) == 1:
                eliminated_idx[week-1] = eliminated_candidates[0]
            elif len(eliminated_candidates) > 1:
                # Check results column first
                if 'results' in season_data.columns:
                    for candidate in eliminated_candidates:
                        result = str(season_data.iloc[candidate]['results'])
                        if f'Eliminated Week {week}' in result or f'eliminated week {week}' in result.lower():
                            eliminated_idx[week-1] = candidate
                            break
                
                # If not found, use placement or other indicators
                if eliminated_idx[week-1] is None and 'placement' in season_data.columns:
                    # Higher placement number means eliminated earlier
                    placements = season_data.loc[eliminated_candidates, 'placement'].values
                    if np.any(~np.isnan(placements)):
                        max_placement = np.nanmax(placements)
                        eliminated_idx[week-1] = eliminated_candidates[
                            np.where(placements == max_placement)[0][0]
                        ]
                
                # Last resort: first candidate
                if eliminated_idx[week-1] is None:
                    eliminated_idx[week-1] = eliminated_candidates[0]
        
        return eliminated_idx

# ==================== 2. IMPROVED OPTIMIZATION MODEL ====================

class ImprovedOptimizationModel:
    """
    Improved optimization model for fan vote estimation
    """
    
    def __init__(self, total_votes=10000000):
        self.all_predictions = {}
        self.total_votes = total_votes
        
    def solve_all_seasons(self, all_seasons_data):
        """Solve for all seasons"""
        print("\nSolving optimization for all seasons...")
        
        for season_num, season_data in all_seasons_data.items():
            print(f"  Season {season_num}...", end=" ", flush=True)
            
            fan_pct = self._solve_single_season_optimized(season_data)
            self.all_predictions[season_num] = fan_pct
            
            consistency = self._calculate_consistency(fan_pct, season_data)
            print(f"Consistency: {consistency:.1%}")
        
        print(f"\n✓ Completed predictions for {len(self.all_predictions)} seasons")
        return self.all_predictions
    
    def _solve_single_season_optimized(self, season_data):
        """Optimized solution for a single season"""
        judge_pct = season_data['judge_data']['percentages']
        judge_ranks = season_data['judge_data']['ranks']
        active_mask = season_data['judge_data']['active_mask']
        eliminated_idx = season_data['elimination_data']
        rule_type = season_data['rule_type']
        features = season_data['features']['matrix']
        n_contestants, n_weeks = judge_pct.shape
        
        fan_pct = np.zeros((n_contestants, n_weeks))
        
        for t in range(n_weeks):
            active_this_week = np.where(active_mask[:, t])[0]
            n_active = len(active_this_week)
            
            if n_active == 0:
                continue
            
            if eliminated_idx[t] is None or eliminated_idx[t] not in active_this_week:
                fan_pct[active_this_week, t] = 1.0 / n_active
                continue
            
            e_t = eliminated_idx[t]
            e_t_idx_in_active = np.where(active_this_week == e_t)[0][0]
            
            if rule_type == 'percentage':
                fan_pct_week = self._solve_percentage_method(
                    judge_pct[active_this_week, t], 
                    e_t_idx_in_active,
                    features[active_this_week, :],
                    n_active
                )
                fan_pct[active_this_week, t] = fan_pct_week
            
            elif rule_type == 'rank':
                fan_pct_week = self._solve_rank_method(
                    judge_ranks[active_this_week, t],
                    e_t_idx_in_active,
                    features[active_this_week, :],
                    n_active
                )
                fan_pct[active_this_week, t] = fan_pct_week
        
        # Apply smoothing
        fan_pct = self._apply_temporal_smoothing(fan_pct, active_mask)
        
        return fan_pct
    
    def _solve_percentage_method(self, judge_pct_week, eliminated_idx, features, n_active):
        """Solve for percentage method weeks"""
        fan_pct = np.ones(n_active) / n_active
        
        # Feature adjustment
        if features.shape[1] > 1:
            feature_weights = np.sum(features[:, :-1], axis=1)  # Exclude intercept
            if np.std(feature_weights) > 0:
                feature_weights = (feature_weights - np.min(feature_weights)) / (np.max(feature_weights) - np.min(feature_weights))
                fan_pct += 0.15 * feature_weights
        
        # Ensure eliminated contestant has lowest total
        total_pct = judge_pct_week + fan_pct
        eliminated_total = total_pct[eliminated_idx]
        
        if not np.isclose(eliminated_total, np.min(total_pct)):
            fan_pct[eliminated_idx] *= 0.5
            
            others = [i for i in range(n_active) if i != eliminated_idx]
            if others:
                increase = (fan_pct[eliminated_idx] * 0.5) / len(others)
                fan_pct[others] += increase
        
        # Apply constraints
        fan_pct = np.maximum(fan_pct, 0.01)
        fan_pct = fan_pct / np.sum(fan_pct)
        
        return fan_pct
    
    def _solve_rank_method(self, judge_ranks_week, eliminated_idx, features, n_active):
        """Solve for rank method weeks"""
        max_rank = np.max(judge_ranks_week[judge_ranks_week > 0])
        judge_scores = max_rank + 1 - judge_ranks_week
        judge_scores = judge_scores / np.sum(judge_scores)
        
        fan_pct = np.ones(n_active) / n_active
        
        # Feature adjustment
        if features.shape[1] > 1:
            feature_weights = np.sum(features[:, :-1], axis=1)
            if np.std(feature_weights) > 0:
                feature_weights = (feature_weights - np.min(feature_weights)) / (np.max(feature_weights) - np.min(feature_weights))
                fan_pct += 0.15 * feature_weights
        
        # Ensure eliminated contestant has worst rank
        fan_pct[eliminated_idx] = np.min(fan_pct) * 0.7
        
        # Apply constraints
        fan_pct = np.maximum(fan_pct, 0.01)
        fan_pct = fan_pct / np.sum(fan_pct)
        
        return fan_pct
    
    def _apply_temporal_smoothing(self, fan_pct, active_mask):
        """Apply temporal smoothing to fan percentages"""
        n_contestants, n_weeks = fan_pct.shape
        
        smoothed = fan_pct.copy()
        
        for i in range(n_contestants):
            active_weeks = np.where(active_mask[i, :])[0]
            if len(active_weeks) < 3:
                continue
            
            for j in range(1, len(active_weeks) - 1):
                week_idx = active_weeks[j]
                prev_week = active_weeks[j-1]
                next_week = active_weeks[j+1]
                
                smoothed[i, week_idx] = 0.7 * fan_pct[i, week_idx] + 0.15 * fan_pct[i, prev_week] + 0.15 * fan_pct[i, next_week]
        
        # Renormalize each week
        for t in range(n_weeks):
            active_this_week = np.where(active_mask[:, t])[0]
            if len(active_this_week) > 0:
                smoothed[active_this_week, t] = smoothed[active_this_week, t] / np.sum(smoothed[active_this_week, t])
        
        return smoothed
    
    def _calculate_consistency(self, fan_pct, season_data):
        """Calculate consistency of predictions"""
        judge_pct = season_data['judge_data']['percentages']
        judge_ranks = season_data['judge_data']['ranks']
        eliminated_idx = season_data['elimination_data']
        active_mask = season_data['judge_data']['active_mask']
        rule_type = season_data['rule_type']
        
        correct = 0
        total = 0
        
        for t in range(fan_pct.shape[1]):
            if eliminated_idx[t] is None:
                continue
            
            active_this_week = np.where(active_mask[:, t])[0]
            if len(active_this_week) <= 1:
                continue
            
            e_t = eliminated_idx[t]
            
            if rule_type == 'percentage':
                total_pct = judge_pct[active_this_week, t] + fan_pct[active_this_week, t]
                pred_elim = active_this_week[np.argmin(total_pct)]
                if pred_elim == e_t:
                    correct += 1
            
            elif rule_type == 'rank':
                fan_pct_week = fan_pct[active_this_week, t]
                pred_elim = active_this_week[np.argmin(fan_pct_week)]
                if pred_elim == e_t:
                    correct += 1
            
            total += 1
        
        return correct / total if total > 0 else 0.0

# ==================== 3. RESULTS SAVER WITH PROPER MERGING ====================

class ResultSaver:
    """Save results to files with proper merging"""
    
    def __init__(self, output_dir="fan_vote_predictions"):
        self.output_dir = output_dir
        os.makedirs(output_dir, exist_ok=True)
        os.makedirs(os.path.join(output_dir, "detailed"), exist_ok=True)
        
    def save_all_results(self, all_predictions, all_seasons_data, original_data):
        """Save all results with proper merging"""
        print("\nSaving results...")
        
        # 1. Create updated data with predictions
        updated_data = self._merge_predictions_with_original(
            all_predictions, all_seasons_data, original_data
        )
        
        # 2. Save updated complete data
        self._save_updated_data(updated_data)
        
        # 3. Save detailed predictions
        self._save_detailed_predictions(all_predictions, all_seasons_data)
        
        # 4. Save summary report
        self._save_summary_report(all_predictions, all_seasons_data)
        
        print(f"\n✓ All results saved to {self.output_dir}/")
        
        return updated_data
    
    def _merge_predictions_with_original(self, all_predictions, all_seasons_data, original_data):
        """Merge predictions back to original data"""
        print("  Merging predictions with original data...")
        
        # Create copy of original data
        updated_data = original_data.copy()
        
        # Ensure unique index for matching
        updated_data.reset_index(drop=True, inplace=True)
        
        # Add prediction columns for each season
        for season_num, fan_pct in all_predictions.items():
            if season_num not in all_seasons_data:
                continue
            
            season_data = all_seasons_data[season_num]
            n_weeks = fan_pct.shape[1]
            
            # Get indices for this season in original data
            season_mask = updated_data['season'] == season_num
            season_indices = updated_data[season_mask].index.tolist()
            
            # Check if indices match
            if len(season_indices) != fan_pct.shape[0]:
                print(f"    ⚠ Season {season_num}: Index mismatch ({len(season_indices)} vs {fan_pct.shape[0]})")
                # Try name-based matching
                contestant_mapping = self._create_contestant_mapping(
                    updated_data[season_mask], season_data
                )
            else:
                # Direct index matching
                contestant_mapping = {i: season_indices[i] for i in range(fan_pct.shape[0])}
            
            print(f"    Season {season_num}: {len(contestant_mapping)} contestants matched")
            
            # Add prediction columns for each week
            for week in range(1, n_weeks + 1):
                col_name_pct = f'Week{week}_Predicted_Fan_Pct'
                col_name_votes = f'Week{week}_Predicted_Fan_Votes'
                
                # Initialize new columns
                if col_name_pct not in updated_data.columns:
                    updated_data[col_name_pct] = np.nan
                if col_name_votes not in updated_data.columns:
                    updated_data[col_name_votes] = np.nan
                
                # Add predictions for each contestant
                for model_idx, original_idx in contestant_mapping.items():
                    if model_idx < fan_pct.shape[0]:
                        if season_data['judge_data']['active_mask'][model_idx, week-1]:
                            fan_pct_value = fan_pct[model_idx, week-1]
                            fan_votes_value = fan_pct_value * 10000000
                            
                            updated_data.at[original_idx, col_name_pct] = fan_pct_value
                            updated_data.at[original_idx, col_name_votes] = fan_votes_value
            
            # Add season summary columns
            for model_idx, original_idx in contestant_mapping.items():
                if model_idx < fan_pct.shape[0]:
                    active_weeks = season_data['judge_data']['active_mask'][model_idx, :]
                    if np.any(active_weeks):
                        avg_pct = np.mean(fan_pct[model_idx, active_weeks])
                        total_votes = avg_pct * 10000000 * np.sum(active_weeks)
                        
                        updated_data.at[original_idx, f'Season{season_num}_Avg_Fan_Pct'] = avg_pct
                        updated_data.at[original_idx, f'Season{season_num}_Total_Fan_Votes'] = total_votes
                        updated_data.at[original_idx, f'Season{season_num}_Weeks_Active'] = np.sum(active_weeks)
        
        return updated_data
    
    def _create_contestant_mapping(self, original_season_data, season_data_info):
        """Create contestant mapping (via name matching)"""
        mapping = {}
        
        # Get contestant info from original data
        original_contestants = {}
        for idx, row in original_season_data.iterrows():
            name = row['celebrity_name'] if 'celebrity_name' in row else None
            if name:
                original_contestants[name] = idx
        
        # Get contestant info from model data
        model_contestants = []
        if 'contestant_info' in season_data_info:
            for info in season_data_info['contestant_info']:
                model_contestants.append(info['name'])
        elif 'original_data' in season_data_info:
            for idx, row in season_data_info['original_data'].iterrows():
                name = row['celebrity_name'] if 'celebrity_name' in row else f'Contestant_{idx}'
                model_contestants.append(name)
        
        # Perform name matching
        for model_idx, model_name in enumerate(model_contestants):
            if model_name in original_contestants:
                mapping[model_idx] = original_contestants[model_name]
            else:
                # Try fuzzy matching
                for orig_name, orig_idx in original_contestants.items():
                    if self._names_match(model_name, orig_name):
                        mapping[model_idx] = orig_idx
                        break
        
        return mapping
    
    def _names_match(self, name1, name2):
        """Check if two names match (simple version)"""
        name1_clean = str(name1).lower().strip()
        name2_clean = str(name2).lower().strip()
        
        # Exact match
        if name1_clean == name2_clean:
            return True
        
        # Partial match
        if name1_clean in name2_clean or name2_clean in name1_clean:
            return True
        
        return False
    
    def _save_updated_data(self, updated_data):
        """Save updated data"""
        # Save as Excel
        excel_file = os.path.join(self.output_dir, "merged_data_with_predictions.xlsx")
        updated_data.to_excel(excel_file, index=False)
        print(f"  ✓ Updated data saved to: {excel_file}")
        
        # Save as CSV (if needed)
        csv_file = os.path.join(self.output_dir, "merged_data_with_predictions.csv")
        updated_data.to_csv(csv_file, index=False)
        print(f"  ✓ CSV version saved to: {csv_file}")
        
        # Show data preview
        print(f"\n  Preview of updated data:")
        print(f"    Total rows: {len(updated_data)}")
        print(f"    Total columns: {len(updated_data.columns)}")
        
        # Show prediction columns
        pred_cols = [col for col in updated_data.columns if 'Predicted' in col]
        print(f"    Prediction columns added: {len(pred_cols)}")
        
        # Show first few rows
        print(f"\n  Sample of predictions (first 5 rows, showing key columns):")
        sample_cols = ['season', 'celebrity_name']
        if len(pred_cols) > 0:
            sample_cols.extend(pred_cols[:3])  # Only show first 3 prediction columns
        
        sample = updated_data[sample_cols].head()
        print(sample.to_string())
        
        # Check for duplicates
        duplicates = updated_data.duplicated(subset=['season', 'celebrity_name']).sum()
        if duplicates > 0:
            print(f"\n  ⚠ Warning: {duplicates} duplicate records found in final data")
    
    def _save_detailed_predictions(self, all_predictions, all_seasons_data):
        """Save detailed predictions"""
        print("\n  Saving detailed predictions...")
        
        for season_num, fan_pct in all_predictions.items():
            if season_num not in all_seasons_data:
                continue
            
            season_data = all_seasons_data[season_num]
            rows = []
            n_contestants, n_weeks = fan_pct.shape
            
            for i in range(n_contestants):
                # Get contestant info
                contestant_name = f"Contestant_{i}"
                if 'contestant_info' in season_data and i < len(season_data['contestant_info']):
                    contestant_name = season_data['contestant_info'][i]['name']
                elif 'original_data' in season_data and i < len(season_data['original_data']):
                    contestant_name = season_data['original_data'].iloc[i]['celebrity_name'] if 'celebrity_name' in season_data['original_data'].columns else f"Contestant_{i}"
                
                for t in range(n_weeks):
                    if season_data['judge_data']['active_mask'][i, t]:
                        row = {
                            'season': season_num,
                            'week': t + 1,
                            'contestant_id': i,
                            'contestant_name': str(contestant_name)[:50],
                            'predicted_fan_percentage': fan_pct[i, t],
                            'predicted_fan_votes': fan_pct[i, t] * 10000000,
                            'rule_type': season_data['rule_type'],
                            'judge_score': season_data['judge_data']['scores'][i, t],
                            'judge_percentage': season_data['judge_data']['percentages'][i, t],
                            'active': True
                        }
                        rows.append(row)
            
            if rows:
                df = pd.DataFrame(rows)
                filename = os.path.join(self.output_dir, "detailed", f"season_{season_num:02d}_detailed.csv")
                df.to_csv(filename, index=False)
                print(f"    ✓ Season {season_num}: {len(rows)} detailed records saved")
    
    def _save_summary_report(self, all_predictions, all_seasons_data):
        """Save summary report"""
        summary_rows = []
        
        for season_num, fan_pct in all_predictions.items():
            if season_num not in all_seasons_data:
                continue
            
            season_data = all_seasons_data[season_num]
            n_contestants, n_weeks = fan_pct.shape
            
            # Calculate season statistics
            total_active_weeks = 0
            total_predicted_votes = 0
            
            for i in range(n_contestants):
                active_weeks = np.sum(season_data['judge_data']['active_mask'][i, :])
                total_active_weeks += active_weeks
                if active_weeks > 0:
                    avg_pct = np.mean(fan_pct[i, season_data['judge_data']['active_mask'][i, :]])
                    total_predicted_votes += avg_pct * 10000000 * active_weeks
            
            summary_rows.append({
                'season': season_num,
                'rule_type': season_data['rule_type'],
                'n_contestants': n_contestants,
                'n_weeks': n_weeks,
                'total_active_weeks': total_active_weeks,
                'total_predicted_votes': total_predicted_votes,
                'avg_votes_per_week': total_predicted_votes / total_active_weeks if total_active_weeks > 0 else 0
            })
        
        if summary_rows:
            df = pd.DataFrame(summary_rows)
            filename = os.path.join(self.output_dir, "season_summary.csv")
            df.to_csv(filename, index=False)
            print(f"  ✓ Season summary saved: {filename}")

# ==================== 4. MAIN PIPELINE ====================

class FanVoteEstimationPipeline:
    """Complete pipeline for fan vote estimation"""
    
    def __init__(self, data_file, total_votes=10000000):
        self.data_file = data_file
        self.total_votes = total_votes
        self.processor = None
        self.model = None
        self.saver = None
        self.original_data = None
        self.results = {}
        
    def run(self):
        """Run complete pipeline"""
        print("=" * 80)
        print("2026 MCM PROBLEM C - FAN VOTE ESTIMATION")
        print(f"Assumed total votes per week: {self.total_votes:,}")
        print("=" * 80)
        
        try:
            # 1. Data Preparation
            print("\n1. DATA PREPARATION")
            print("-" * 40)
            self.processor = EnhancedDataProcessor(self.data_file)
            self.original_data = self.processor.load_and_clean_data()
            
            if self.original_data is None:
                raise ValueError("Failed to load data")
            
            self.processor.determine_season_rules_enhanced()
            all_seasons_data = self.processor.prepare_all_seasons_data()
            
            # 2. Modeling
            print("\n2. MODELING - FAN VOTE ESTIMATION")
            print("-" * 40)
            self.model = ImprovedOptimizationModel(total_votes=self.total_votes)
            all_predictions = self.model.solve_all_seasons(all_seasons_data)
            
            # 3. Save Results
            print("\n3. SAVING RESULTS")
            print("-" * 40)
            self.saver = ResultSaver()
            updated_data = self.saver.save_all_results(
                all_predictions, all_seasons_data, self.original_data
            )
            
            # Store results
            self.results = {
                'predictions': all_predictions,
                'all_seasons_data': all_seasons_data,
                'updated_data': updated_data
            }
            
            # Final summary
            self._print_final_summary(updated_data)
            
            return self.results
            
        except Exception as e:
            print(f"\n❌ Error: {e}")
            import traceback
            traceback.print_exc()
            return None
    
    def _print_final_summary(self, updated_data):
        """Print final summary"""
        print("\n" + "=" * 80)
        print("ANALYSIS COMPLETE - FINAL SUMMARY")
        print("=" * 80)
        
        print(f"\n📊 DATA SUMMARY:")
        print(f"  Total records: {len(updated_data)}")
        print(f"  Seasons analyzed: {len(updated_data['season'].unique())}")
        
        # Show prediction columns
        pred_pct_cols = [col for col in updated_data.columns if 'Predicted_Fan_Pct' in col]
        pred_votes_cols = [col for col in updated_data.columns if 'Predicted_Fan_Votes' in col]
        
        print(f"\n📈 PREDICTIONS GENERATED:")
        print(f"  Fan percentage columns: {len(pred_pct_cols)}")
        print(f"  Fan vote columns: {len(pred_votes_cols)}")
        
        if pred_pct_cols:
            # Show sample data
            sample_data = updated_data[['season', 'celebrity_name'] + pred_pct_cols[:3]].head(10)
            print(f"\n📋 SAMPLE OF PREDICTIONS (First 10 rows):")
            print(sample_data.to_string(index=False))
            
            # Check prediction validity
            print(f"\n✅ PREDICTION VALIDATION:")
            for col in pred_pct_cols[:2]:  # Check first two weeks
                if col in updated_data.columns:
                    valid_pct = updated_data[col].notna().sum()
                    avg_pct = updated_data[col].mean()
                    print(f"  {col}: {valid_pct} valid predictions, average: {avg_pct:.2%}")
        
        print(f"\n📁 OUTPUT FILES:")
        print(f"  1. Main file with predictions: fan_vote_predictions/merged_data_with_predictions.xlsx")
        print(f"  2. Detailed predictions: fan_vote_predictions/detailed/season_XX_detailed.csv")
        print(f"  3. Season summary: fan_vote_predictions/season_summary.csv")
        
        print(f"\n💡 NEXT STEPS:")
        print(f"  1. Open 'merged_data_with_predictions.xlsx' to see all predictions")
        print(f"  2. Use the predicted fan votes for Tasks 2-4 of the problem")
        print(f"  3. Check 'season_summary.csv' for overall statistics")

# ==================== 5. MAIN EXECUTION ====================

def main():
    """Main execution function"""
    
    # Configuration
    DATA_FILE = "merged_ranking_percentage.xlsx"  # Your data file
    TOTAL_VOTES = 10000000  # 10 million total votes
    
    print("=" * 80)
    print("2026 MCM PROBLEM C - FAN VOTE ESTIMATION")
    print(f"Assumed total votes per week: {TOTAL_VOTES:,}")
    print("=" * 80)
    
    # Create and run pipeline
    pipeline = FanVoteEstimationPipeline(DATA_FILE, TOTAL_VOTES)
    results = pipeline.run()
    
    if results and 'updated_data' in results:
        print("\n" + "=" * 80)
        print("✅ FAN VOTE ESTIMATION COMPLETED SUCCESSFULLY!")
        print("=" * 80)
        
        # Show key information
        updated_data = results['updated_data']
        
        # Check data quality
        print(f"\n🔍 DATA QUALITY CHECK:")
        print(f"  Total unique contestants: {updated_data[['season', 'celebrity_name']].drop_duplicates().shape[0]}")
        
        # Check duplicates
        duplicates = updated_data.duplicated(subset=['season', 'celebrity_name']).sum()
        if duplicates == 0:
            print(f"  ✓ No duplicate contestants found")
        else:
            print(f"  ⚠ {duplicates} duplicate contestants found")
        
        # Show file location
        import os
        abs_path = os.path.abspath("fan_vote_predictions/merged_data_with_predictions.xlsx")
        print(f"\n📂 Main output file location:")
        print(f"  {abs_path}")
    else:
        print("\n" + "=" * 80)
        print("❌ ANALYSIS FAILED")
        print("=" * 80)

if __name__ == "__main__":
    main()

2nd problem

In [ ]:

print("="*80)
print("COMPLETE DWTS JUDGE VS FAN PERCENTAGE ANALYSIS")
print("="*80)

import pandas as pd
import numpy as np
import warnings
import sys

warnings.filterwarnings('ignore')

# 1. Load the original data
print("\n1. Loading original DWTS data...")
df = pd.read_csv('2026_MCM_Problem_C_Data.csv', encoding='utf-8')
print(f"   Loaded {len(df)} records from the original dataset")

# 2. Clean and prepare the data
print("\n2. Cleaning and preparing data...")

# Convert N/A values to NaN
df = df.replace('N/A', np.nan)

# Create a clean copy for calculations
df_clean = df.copy()

# 3. Calculate total judge scores for each contestant
print("\n3. Calculating total judge scores...")

# Find all judge score columns
judge_columns = [col for col in df_clean.columns if 'judge' in col.lower() and 'score' in col.lower()]

# Convert all judge score columns to numeric
for col in judge_columns:
    df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

# Calculate total judge score for each contestant
df_clean['total_judge_score'] = df_clean[judge_columns].sum(axis=1)

print(f"   Found {len(judge_columns)} judge score columns")
print(f"   Total judge scores calculated for {len(df_clean)} contestants")

# 4. Calculate judge percentages
print("\n4. Calculating judge percentages...")
total_all_judge_scores = df_clean['total_judge_score'].sum()
df_clean['judge_percentage'] = (df_clean['total_judge_score'] / total_all_judge_scores) * 100

# 5. Estimate fan votes (this is a critical modeling step)
print("\n5. Estimating fan votes based on placement and results...")

# First, let's estimate fan votes using several factors
df_clean['estimated_fan_votes'] = 0

# Create a scoring system based on multiple factors
for idx, row in df_clean.iterrows():
    fan_score = 0
    
    # 1. Base on final placement (better placement = more fans)
    if pd.notna(row['placement']):
        try:
            placement = float(row['placement'])
            # Higher placement (lower number) gets more points
            fan_score += (11 - min(placement, 10)) * 1000  # Max 10,000 points
        except:
            pass
    
    # 2. Bonus for winners and finalists
    results_str = str(row['results']).lower()
    if '1st' in results_str or 'winner' in results_str:
        fan_score += 5000
    elif '2nd' in results_str:
        fan_score += 4000
    elif '3rd' in results_str:
        fan_score += 3000
    elif 'final' in results_str or 'finalist' in results_str:
        fan_score += 2000
    
    # 3. Consider how long they lasted in the competition
    # Extract week number from results if possible
    results_text = str(row['results']).lower()
    if 'week' in results_text:
        try:
            # Find the week number
            for word in results_text.split():
                if word.isdigit():
                    week_num = int(word)
                    # More weeks = more exposure = more fans
                    fan_score += week_num * 500
                    break
        except:
            pass
    
    # 4. Age factor (younger contestants might have more social media fans)
    if pd.notna(row['celebrity_age_during_season']):
        age = float(row['celebrity_age_during_season'])
        # Younger contestants get a slight bonus (simulating social media following)
        if age < 30:
            fan_score += 1000
        elif age < 50:
            fan_score += 500
    
    # 5. Industry factor (some industries have larger fan bases)
    industry = str(row['celebrity_industry']).lower()
    if 'actor' in industry or 'actress' in industry:
        fan_score += 2000
    elif 'singer' in industry or 'rapper' in industry:
        fan_score += 2500  # Music artists often have strong fan bases
    elif 'athlete' in industry:
        fan_score += 1500
    elif 'tv' in industry or 'reality' in industry:
        fan_score += 3000  # TV personalities often have dedicated fan bases
    
    # 6. Season number (later seasons had more viewers)
    if pd.notna(row['season']):
        try:
            season_num = float(row['season'])
            # Later seasons had more viewers
            fan_score += min(season_num, 30) * 100
        except:
            pass
    
    # Add some randomness to simulate real-world variation
    np.random.seed(idx)  # For reproducibility
    fan_score += np.random.randint(0, 2000)
    
    # Ensure minimum fan score
    fan_score = max(fan_score, 100)
    
    df_clean.at[idx, 'estimated_fan_votes'] = fan_score

print(f"   Fan votes estimated for {len(df_clean)} contestants")

# 6. Calculate fan percentages
print("\n6. Calculating fan percentages...")
total_all_fan_votes = df_clean['estimated_fan_votes'].sum()
df_clean['fan_percentage'] = (df_clean['estimated_fan_votes'] / total_all_fan_votes) * 100

# 7. Calculate total percentage (judge + fan)
print("\n7. Calculating total percentages...")
df_clean['total_percentage'] = df_clean['judge_percentage'] + df_clean['fan_percentage']

# 8. Sort by total percentage (highest to lowest)
print("\n8. Sorting by total percentage...")
df_clean = df_clean.sort_values('total_percentage', ascending=False)

# 9. Save the complete percentage data
print("\n9. Saving complete percentage data...")
percentage_df = df_clean[['celebrity_name', 'total_judge_score', 'judge_percentage', 
                         'estimated_fan_votes', 'fan_percentage', 'total_percentage']].copy()

percentage_df.to_csv('DWTS_Judge_Fan_Percentage_Data.csv', index=False, encoding='utf-8')
print(f"   ✓ Complete percentage data saved: DWTS_Judge_Fan_Percentage_Data.csv")
print(f"   Contains {len(percentage_df)} contestants")

# ======================================================================
# PART 2: CREATE SIMPLIFIED TABLE
# ======================================================================

print("\n" + "="*80)
print("CREATING SIMPLIFIED JUDGE VS FAN PERCENTAGE TABLE")
print("="*80)

# 10. Select only required columns for simplified table
print("\n10. Selecting required columns for simplified table...")

required_columns = [
    'celebrity_name',
    'total_judge_score', 
    'judge_percentage',
    'estimated_fan_votes',
    'fan_percentage',
    'total_percentage'
]

simplified_df = percentage_df[required_columns].copy()

# 11. Format the data
print("\n11. Formatting data...")

# Keep numeric values for calculations
numeric_df = simplified_df.copy()

# Create display version with formatted percentages
display_df = simplified_df.copy()

# Format percentages to 2 decimal places
display_df['judge_percentage'] = display_df['judge_percentage'].apply(lambda x: f"{x:.2f}%")
display_df['fan_percentage'] = display_df['fan_percentage'].apply(lambda x: f"{x:.2f}%")
display_df['total_percentage'] = display_df['total_percentage'].apply(lambda x: f"{x:.2f}%")

# Format numbers with commas
display_df['total_judge_score'] = display_df['total_judge_score'].apply(lambda x: f"{x:,.0f}")
display_df['estimated_fan_votes'] = display_df['estimated_fan_votes'].apply(lambda x: f"{x:,.0f}")

# 12. Add total row
print("\n12. Adding total row...")

# Calculate totals
total_judge_score = numeric_df['total_judge_score'].sum()
total_fan_votes = numeric_df['estimated_fan_votes'].sum()

# For percentages, we need to calculate properly
total_judge_percentage = 100.00
total_fan_percentage = 100.00
total_combined_percentage = 200.00

# Create total row for numeric data
total_row_numeric = pd.DataFrame([{
    'celebrity_name': 'TOTAL',
    'total_judge_score': total_judge_score,
    'judge_percentage': total_judge_percentage,
    'estimated_fan_votes': total_fan_votes,
    'fan_percentage': total_fan_percentage,
    'total_percentage': total_combined_percentage
}])

# Create total row for display data
total_row_display = pd.DataFrame([{
    'celebrity_name': 'TOTAL',
    'total_judge_score': f"{total_judge_score:,.0f}",
    'judge_percentage': f"{total_judge_percentage:.2f}%",
    'estimated_fan_votes': f"{total_fan_votes:,.0f}",
    'fan_percentage': f"{total_fan_percentage:.2f}%",
    'total_percentage': f"{total_combined_percentage:.2f}%"
}])

# 13. Combine data with totals
print("\n13. Combining data with totals...")

final_numeric_df = pd.concat([numeric_df, total_row_numeric], ignore_index=True)
final_display_df = pd.concat([display_df, total_row_display], ignore_index=True)

# 14. Rename columns for clarity
print("\n14. Renaming columns...")

column_names = {
    'celebrity_name': 'Player Name',
    'total_judge_score': 'Total Judge Score',
    'judge_percentage': 'Judge Score %',
    'estimated_fan_votes': 'Estimated Fan Votes',
    'fan_percentage': 'Fan Vote %', 
    'total_percentage': 'Total %'
}

final_numeric_df = final_numeric_df.rename(columns=column_names)
final_display_df = final_display_df.rename(columns=column_names)

# 15. Save the files
print("\n15. Saving simplified tables...")

# Save numeric version (for calculations)
final_numeric_df.to_csv('DWTS_Simplified_Percentage_Table_Numeric.csv', index=False, encoding='utf-8')
print(f"   ✓ Numeric table saved: DWTS_Simplified_Percentage_Table_Numeric.csv")

# Save display version (for viewing)
final_display_df.to_csv('DWTS_Simplified_Percentage_Table_Display.csv', index=False, encoding='utf-8')
print(f"   ✓ Display table saved: DWTS_Simplified_Percentage_Table_Display.csv")

# 16. Display sample of the table
print("\n16. Sample of the simplified table (first 10 rows + total):")
print("-" * 90)

# Display first 10 rows + total row
sample_display = pd.concat([final_display_df.head(10), final_display_df.tail(1)], ignore_index=True)

pd.set_option('display.width', 120)
pd.set_option('display.max_columns', None)
print(sample_display.to_string(index=False))

# 17. Create summary statistics
print("\n17. Creating summary statistics...")

# Calculate some key metrics from the data (excluding the TOTAL row)
data_without_total = final_numeric_df[final_numeric_df['Player Name'] != 'TOTAL']

summary_stats = {
    'Statistic': [
        'Number of Players',
        'Average Total Judge Score',
        'Average Judge Score %',
        'Average Estimated Fan Votes',
        'Average Fan Vote %',
        'Average Total %',
        'Highest Judge Score %',
        'Highest Fan Vote %',
        'Highest Total %',
        'Median Judge Score %',
        'Median Fan Vote %'
    ],
    'Value': [
        len(data_without_total),
        f"{data_without_total['Total Judge Score'].mean():,.0f}",
        f"{data_without_total['Judge Score %'].mean():.2f}%",
        f"{data_without_total['Estimated Fan Votes'].mean():,.0f}",
        f"{data_without_total['Fan Vote %'].mean():.2f}%",
        f"{data_without_total['Total %'].mean():.2f}%",
        f"{data_without_total['Judge Score %'].max():.2f}%",
        f"{data_without_total['Fan Vote %'].max():.2f}%",
        f"{data_without_total['Total %'].max():.2f}%",
        f"{data_without_total['Judge Score %'].median():.2f}%",
        f"{data_without_total['Fan Vote %'].median():.2f}%"
    ],
    'Player': [
        '',
        data_without_total.loc[data_without_total['Total Judge Score'].idxmax(), 'Player Name'],
        data_without_total.loc[data_without_total['Judge Score %'].idxmax(), 'Player Name'],
        data_without_total.loc[data_without_total['Estimated Fan Votes'].idxmax(), 'Player Name'],
        data_without_total.loc[data_without_total['Fan Vote %'].idxmax(), 'Player Name'],
        data_without_total.loc[data_without_total['Total %'].idxmax(), 'Player Name'],
        data_without_total.loc[data_without_total['Judge Score %'].idxmax(), 'Player Name'],
        data_without_total.loc[data_without_total['Fan Vote %'].idxmax(), 'Player Name'],
        data_without_total.loc[data_without_total['Total %'].idxmax(), 'Player Name'],
        '',
        ''
    ]
}

summary_df = pd.DataFrame(summary_stats)
summary_df.to_csv('DWTS_Percentage_Summary.csv', index=False, encoding='utf-8')
print(f"\n   ✓ Summary statistics saved: DWTS_Percentage_Summary.csv")

# 18. Check controversial players
print("\n18. Analyzing controversial players...")

controversial_players = ['Jerry Rice', 'Bobby Bones', 'Bristol Palin', 'Billy Ray Cyrus']

print("\nControversial players analysis (players where fan support differed significantly from judge scores):")
for player in controversial_players:
    player_data = final_numeric_df[final_numeric_df['Player Name'] == player]
    if len(player_data) > 0:
        row = player_data.iloc[0]
        print(f"\n   {player}:")
        print(f"      Judge Score: {row['Total Judge Score']:,.0f} ({row['Judge Score %']:.2f}%)")
        print(f"      Fan Votes: {row['Estimated Fan Votes']:,.0f} ({row['Fan Vote %']:.2f}%)")
        print(f"      Total: {row['Total %']:.2f}%")
        if row['Judge Score %'] > 0:
            ratio = row['Fan Vote %'] / row['Judge Score %']
            print(f"      Fan/Judge Ratio: {ratio:.2f}:1")

# 19. Create verification check
print("\n19. Data verification:")
print("-" * 50)

# Check if percentages sum correctly
judge_percent_sum = data_without_total['Judge Score %'].sum()
fan_percent_sum = data_without_total['Fan Vote %'].sum()
total_percent_sum = data_without_total['Total %'].sum()

print(f"Sum of Judge Score % (should be ~100%): {judge_percent_sum:.2f}%")
print(f"Sum of Fan Vote % (should be ~100%): {fan_percent_sum:.2f}%")
print(f"Sum of Total % (should be ~200%): {total_percent_sum:.2f}%")

# 20. Quick analysis example
print("\n20. Quick analysis example:")
print("-" * 50)

# Find top 5 players by fan-to-judge ratio
data_without_total['Fan_Judge_Ratio'] = data_without_total['Fan Vote %'] / data_without_total['Judge Score %']
top_ratios = data_without_total.nlargest(5, 'Fan_Judge_Ratio')[['Player Name', 'Judge Score %', 'Fan Vote %', 'Fan_Judge_Ratio']]

print("\nTop 5 players by Fan/Judge Ratio (highest fan influence relative to judges):")
for _, row in top_ratios.iterrows():
    print(f"  {row['Player Name']}: Judge {row['Judge Score %']:.2f}%, Fan {row['Fan Vote %']:.2f}%, Ratio: {row['Fan_Judge_Ratio']:.2f}")

print("\n" + "="*80)
print("ANALYSIS COMPLETE! FILES CREATED:")
print("="*80)
print("""
1. DWTS_Judge_Fan_Percentage_Data.csv - Complete percentage data
2. DWTS_Simplified_Percentage_Table_Numeric.csv - Numeric table for calculations
3. DWTS_Simplified_Percentage_Table_Display.csv - Formatted table for viewing/reports
4. DWTS_Percentage_Summary.csv - Summary statistics

TABLE STRUCTURE:
• Player Name
• Total Judge Score
• Judge Score %
• Estimated Fan Votes  
• Fan Vote %
• Total %
• Final row: TOTAL with sums
""")

print("\nHOW TO USE FOR MCM PROBLEM C:")
print("-" * 50)
print("""
1. Compare Judge Score % vs Fan Vote % for each player
2. Identify controversial cases (high fan, low judge scores)
3. Calculate Fan/Judge ratio to measure fan influence
4. Use the data to model different voting systems:
   - Rank-based system vs Percentage-based system
   - How each system amplifies or diminishes fan influence
5. Test your fan vote estimation model against known controversial cases
""")

print("\nKEY FINDINGS:")
print("-" * 50)

# Calculate overall fan influence
avg_judge_percent = data_without_total['Judge Score %'].mean()
avg_fan_percent = data_without_total['Fan Vote %'].mean()
print(f"Average Judge Score % per player: {avg_judge_percent:.4f}%")
print(f"Average Fan Vote % per player: {avg_fan_percent:.4f}%")
print(f"Average Fan/Judge ratio: {avg_fan_percent/avg_judge_percent:.2f}")

# Count controversial cases (ratio > 2.0)
controversial_count = len(data_without_total[data_without_total['Fan Vote %'] > 2 * data_without_total['Judge Score %']])
print(f"\nPlayers with strong fan support (Fan % > 2× Judge %): {controversial_count} out of {len(data_without_total)}")

print("\n" + "="*80)
print("ANALYSIS READY FOR MCM PROBLEM C SUBMISSION!")
print("="*80)

In [ ]:
import pandas as pd
import numpy as np
import unicodedata

# Load data
df = pd.read_excel("merged_data_with_predictions.xlsx")

def clean(x):
    """Clean and normalize names"""
    x = str(x).lower()
    x = unicodedata.normalize("NFKD", x)
    return "".join(c for c in x if c.isalpha())

df["name_clean"] = df["celebrity_name"].apply(clean)

# Controversial contestants mapping
name_map = {
    2: "Jerry Rice",
    4: "Billy Ray Cyrus",
    11: "Bristol Palin",
    27: "Bobby Bones"
}

# Method mapping for each season
method_map = {
    2: "rank",
    4: "percent",
    11: "percent",
    27: "percent"
}

# Create clean name mapping for controversial contestants
controversial = {s: clean(n) for s,n in name_map.items()}

# Extract week numbers
weeks = sorted({
    int(c.replace("Week","").split("_")[0])
    for c in df.columns if "Predicted_Fan_Votes" in c
})

rows = []

# Process each controversial contestant
for season, star in controversial.items():

    season_df = df[df["season"] == season].copy()
    alive = set(season_df["name_clean"])

    for week in weeks:

        fan_col = f"Week{week}_Predicted_Fan_Votes"
        judge_cols = [c for c in season_df.columns if c.startswith(f"week{week}_judge")]

        if fan_col not in season_df.columns or not judge_cols:
            continue

        alive_df = season_df[season_df["name_clean"].isin(alive)].copy()
        if len(alive_df) <= 1:
            break

        F = alive_df[fan_col]
        J = alive_df[judge_cols].sum(axis=1)

        mask = ~F.isna()
        alive_df = alive_df[mask]
        F = F[mask].values
        J = J[mask].values

        if len(alive_df) <= 1:
            break

        alive_df["judge_sum"] = J
        n = len(alive_df)

        # ===== Ranking calculation =====
        if method_map[season] == "percent":
            # Percentage method: J/J.sum() + F/F.sum()
            S = J/J.sum() + F/F.sum()
            order = np.argsort(-S)
        else:
            # Rank method: rJ + rF
            rJ = pd.Series(J).rank(ascending=False).values
            rF = pd.Series(F).rank(ascending=False).values
            S = rJ + rF
            order = np.argsort(S)

        ranks = np.empty(n)
        ranks[order] = np.arange(1, n+1)
        alive_df["rank"] = ranks

        sorted_df = alive_df.sort_values("rank", ascending=False)
        bottom1 = sorted_df.iloc[0]
        bottom2 = sorted_df.iloc[1]

        if star in alive_df["name_clean"].values:

            star_row = alive_df[alive_df["name_clean"] == star].iloc[0]
            star_rank = int(star_row["rank"])
            is_bottom2 = star_rank >= n - 1

            should_eliminate = False

            if is_bottom2 and bottom2["name_clean"] == star:
                should_eliminate = star_row["judge_sum"] < bottom1["judge_sum"]

            rows.append({
                "season": season,
                "week": week,
                "contestant": star_row["celebrity_name"],
                "method": method_map[season],
                "rank": star_rank,
                "is_bottom2": is_bottom2,
                "bottom1": bottom1["celebrity_name"] if is_bottom2 else None,
                "judge_contestant": star_row["judge_sum"] if is_bottom2 else None,
                "judge_bottom1": bottom1["judge_sum"] if is_bottom2 else None,
                "should_be_eliminated": should_eliminate
            })

        # ===== Advance real elimination =====
        real_out = season_df[season_df[fan_col].isna()]
        if len(real_out) == 1:
            alive.remove(clean(real_out.iloc[0]["celebrity_name"]))

# ======================
# Export results
# ======================

pd.DataFrame(rows).to_excel(
    "controversial_mixed_method_ranking_with_decision.xlsx",
    index=False
)

print("✅ Complete ranking table with theoretical elimination decisions generated")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import unicodedata

# ======================
# Read Data
# ======================

df = pd.read_excel("merged_data_with_predictions.xlsx")

# ======================
# Name Cleaning
# ======================

def clean(x):
    x = str(x).lower()
    x = unicodedata.normalize("NFKD", x)
    return "".join(c for c in x if c.isalpha())

df["name_clean"] = df["celebrity_name"].apply(clean)

# ======================
# Controversial Contestants and Method Selection
# ======================

name_map = {
    2: "Jerry Rice",
    4: "Billy Ray Cyrus", 
    11: "Bristol Palin",
    27: "Bobby Bones"
}

method_map = {
    2: "rank",
    4: "percent", 
    11: "percent",
    27: "percent"
}

controversial = {s: clean(n) for s, n in name_map.items()}

# ======================
# Extract Weeks
# ======================

weeks = sorted({
    int(c.replace("Week", "").split("_")[0])
    for c in df.columns if "Predicted_Fan_Votes" in c
})

# ======================
# Store Results
# ======================

tracks = {}
alive_counts = {}

# ======================
# Main Loop
# ======================

for season, star in controversial.items():

    season_df = df[df["season"] == season].copy()
    alive = set(season_df["name_clean"])

    tracks[season] = []
    alive_counts[season] = []

    for w in weeks:

        fan_col = f"Week{w}_Predicted_Fan_Votes"
        judge_cols = [c for c in season_df.columns if c.startswith(f"week{w}_judge")]

        if fan_col not in season_df.columns or not judge_cols:
            continue

        alive_df = season_df[season_df["name_clean"].isin(alive)].copy()
        if len(alive_df) <= 1:
            break

        F = alive_df[fan_col]
        J = alive_df[judge_cols].sum(axis=1)

        mask = ~F.isna()
        alive_df = alive_df[mask]
        F = F[mask].values
        J = J[mask].values

        if len(alive_df) <= 1:
            break

        n = len(alive_df)
        alive_counts[season].append(n)

        # ======================
        # Calculate Rankings
        # ======================

        if method_map[season] == "percent":
            S = J/J.sum() + F/F.sum()
            order = np.argsort(-S)

        else:  # rank method
            rJ = pd.Series(J).rank(ascending=False).values
            rF = pd.Series(F).rank(ascending=False).values
            S = rJ + rF
            order = np.argsort(S)

        ranks = np.empty(n)
        ranks[order] = np.arange(1, n+1)

        # ======================
        # Record Controversial Contestant
        # ======================

        if star in alive_df["name_clean"].values:
            idx = list(alive_df["name_clean"]).index(star)
            tracks[season].append(ranks[idx])
        else:
            break

        # ======================
        # Advance Real Elimination
        # ======================

        real_out = season_df[season_df[fan_col].isna()]
        if len(real_out) == 1:
            alive.remove(clean(real_out.iloc[0]["celebrity_name"]))

# ======================
# Plotting (Bottom 2 Highlight)
# ======================

plt.figure(figsize=(11, 6))

for season, y in tracks.items():

    name = name_map[season]
    x = range(1, len(y)+1)

    plt.plot(x, y, marker="o", linewidth=2, label=name)

    for i, rank in enumerate(y):
        n = alive_counts[season][i]
        bottom2_start = n - 1

        # Red danger zone
        plt.axhspan(bottom2_start, n,
                    xmin=i/len(y), xmax=(i+1)/len(y),
                    color="red", alpha=0.18)

        # Red dots
        if rank >= bottom2_start:
            plt.scatter(i+1, rank, s=90, color="red", zorder=5)

plt.gca().invert_yaxis()
plt.xlabel("Week")
plt.ylabel("Rank (1 = Best)")
plt.title("Controversial Contestants – Best-Fit Ranking with Bottom 2 Highlighted")
plt.legend()
plt.tight_layout()
plt.show()

3rd problem

In [ ]:
# =========================
# 1. Import libraries and data
# =========================
import pandas as pd
import numpy as np
import unicodedata
import matplotlib.pyplot as plt

# Set plot style
plt.rcParams['font.sans-serif'] = ['Arial', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# Load data
print("Loading data...")
raw = pd.read_csv("2026_MCM_Problem_C_Data.csv")
pred = pd.read_excel("merged_data_with_predictions.xlsx")

print(f"Raw data shape: {raw.shape}")
print(f"Prediction data shape: {pred.shape}")

# =========================
# 2. Data cleaning and merging
# =========================

# Name cleaning function
def clean_name(x):
    """Clean and normalize names"""
    x = str(x).lower()
    x = unicodedata.normalize("NFKD", x)
    return "".join(c for c in x if c.isalpha())

print("\nCleaning names...")
raw["name_clean"] = raw["celebrity_name"].apply(clean_name)
pred["name_clean"] = pred["celebrity_name"].apply(clean_name)

# Industry reclassification function
def categorize_industry(industry):
    """Categorize industries into standardized groups"""
    if pd.isna(industry):
        return "Other"
    
    industry = str(industry).lower()
    
    # Actor category
    actor_keywords = ["actor", "actress", "actor/actress"]
    if any(keyword in industry for keyword in actor_keywords):
        return "Actor/Actress"
    
    # Athlete category
    athlete_keywords = ["athlete", "sports", "racing"]
    if any(keyword in industry for keyword in athlete_keywords):
        return "Athlete"
    
    # Singer/idol category
    music_keywords = ["singer", "rapper", "musician"]
    if any(keyword in industry for keyword in music_keywords):
        return "Singer/Musician"
    
    # Model category
    if "model" in industry:
        return "Model"
    
    # TV personality
    tv_keywords = ["tv personality", "reality", "social media"]
    if any(keyword in industry for keyword in tv_keywords):
        return "TV Personality"
    
    # Other
    return "Other"

print("Categorizing occupations...")
raw["occupation_category"] = raw["celebrity_industry"].apply(categorize_industry)

# Check data columns
print("\nRaw data columns:")
print(raw.columns.tolist())

# Find age column
age_col = None
for col in raw.columns:
    if 'age' in col.lower():
        age_col = col
        break

print(f"\nAge column found: {age_col}")

# Merge data
print("Merging data...")
if age_col:
    industry_map = raw[["name_clean", "occupation_category", age_col]].drop_duplicates()
    industry_map = industry_map.rename(columns={age_col: "age"})
else:
    industry_map = raw[["name_clean", "occupation_category"]].drop_duplicates()
    industry_map["age"] = np.nan

df = pred.merge(industry_map, on="name_clean", how="left")
print(f"Merged data shape: {df.shape}")

# =========================
# 3. Extract weekly data
# =========================

print("\nExtracting weekly data...")
week_cols = [c for c in df.columns if "Predicted_Fan_Votes" in c]
weeks = []

for col in week_cols:
    try:
        week_num = int(col.replace("Week", "").split("_")[0])
        weeks.append(week_num)
    except:
        continue

weeks = sorted(set(weeks))
print(f"Weeks found: {weeks}")

# =========================
# 4. Calculate weekly rankings
# =========================

print("\nCalculating weekly rankings...")
all_data = []

for w in weeks:
    fan_col = f"Week{w}_Predicted_Fan_Votes"
    
    if fan_col not in df.columns:
        continue
    
    # Find judge score columns
    judge_patterns = [f"week{w}_judge", f"Week{w}_Judge"]
    judge_cols = []
    
    for pattern in judge_patterns:
        judge_cols = [c for c in df.columns if c.startswith(pattern)]
        if judge_cols:
            break
    
    if not judge_cols:
        continue
    
    # Filter rows with data
    week_data = df[~df[fan_col].isna()].copy()
    
    if len(week_data) < 2:
        continue
    
    # Calculate totals and rankings
    week_data["judge_total"] = week_data[judge_cols].sum(axis=1)
    week_data["fan_votes"] = week_data[fan_col]
    
    week_data["judge_rank"] = week_data["judge_total"].rank(ascending=False)
    week_data["fan_rank"] = week_data["fan_votes"].rank(ascending=False)
    
    week_data["week"] = w
    
    # Select required columns
    keep_cols = ["occupation_category", "age", "judge_rank", "fan_rank", "week", "celebrity_name"]
    all_data.append(week_data[keep_cols])

# Check if data exists
if not all_data:
    print("Error: No valid data found!")
    exit()

# Combine all data
all_rankings = pd.concat(all_data, ignore_index=True)
print(f"\nTotal data points: {len(all_rankings)}")

# =========================
# 5. Analyze occupation performance
# =========================

print("\nAnalyzing occupation performance...")
occupation_stats = all_rankings.groupby("occupation_category").agg({
    "judge_rank": ["mean", "count"],
    "fan_rank": ["mean", "count"]
}).round(2)

print("\n" + "="*60)
print("Average Ranking Statistics by Occupation")
print("="*60)
print(occupation_stats)

# =========================
# 6. Analyze age performance
# =========================

print("\nAnalyzing age performance...")
all_rankings["age_numeric"] = pd.to_numeric(all_rankings["age"], errors='coerce')
age_data = all_rankings[~all_rankings["age_numeric"].isna()].copy()

if len(age_data) > 0:
    # Create age groups
    age_data["age_group"] = pd.cut(age_data["age_numeric"], 
                                  bins=[0, 20, 30, 40, 50, 60, 100],
                                  labels=["<20", "20-29", "30-39", "40-49", "50-59", "60+"])
    
    age_stats = age_data.groupby("age_group").agg({
        "judge_rank": ["mean", "count"],
        "fan_rank": ["mean", "count"]
    }).round(2)
    
    print("\n" + "="*60)
    print("Performance Statistics by Age Group")
    print("="*60)
    print(age_stats)
    
    has_age_data = True
else:
    print("No valid age data")
    has_age_data = False

# =========================
# 7. Create visualizations
# =========================

print("\nCreating visualizations...")

# Prepare data
occupations = occupation_stats.index.tolist()
judge_means = occupation_stats[("judge_rank", "mean")]
fan_means = occupation_stats[("fan_rank", "mean")]

# Create charts
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle("DWTS Performance Analysis by Occupation and Age", fontsize=16, fontweight='bold')

# Subplot 1: Occupation performance comparison
ax1 = axes[0, 0]
x = np.arange(len(occupations))
width = 0.35

bars1 = ax1.bar(x - width/2, judge_means, width, label='Judge', color='blue', alpha=0.7)
bars2 = ax1.bar(x + width/2, fan_means, width, label='Fan', color='red', alpha=0.7)

ax1.set_xlabel('Occupation')
ax1.set_ylabel('Average Rank (Lower = Better)')
ax1.set_title('Judge vs Fan Rankings by Occupation')
ax1.set_xticks(x)
ax1.set_xticklabels(occupations, rotation=45, ha='right')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Add values
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.1f}', ha='center', va='bottom', fontsize=9)

# Subplot 2: Judge rankings
ax2 = axes[0, 1]
colors = plt.cm.tab20c(np.linspace(0, 1, len(occupations)))
bars_judge = ax2.bar(occupations, judge_means, color=colors)
ax2.set_xlabel('Occupation')
ax2.set_ylabel('Average Judge Rank')
ax2.set_title('Judge Rankings by Occupation')
ax2.set_xticklabels(occupations, rotation=45, ha='right')
ax2.grid(True, alpha=0.3)

for bar in bars_judge:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.1f}', ha='center', va='bottom', fontsize=9)

# Subplot 3: Fan rankings
ax3 = axes[1, 0]
bars_fan = ax3.bar(occupations, fan_means, color=colors)
ax3.set_xlabel('Occupation')
ax3.set_ylabel('Average Fan Rank')
ax3.set_title('Fan Rankings by Occupation')
ax3.set_xticklabels(occupations, rotation=45, ha='right')
ax3.grid(True, alpha=0.3)

for bar in bars_fan:
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.1f}', ha='center', va='bottom', fontsize=9)

# Subplot 4: Ranking differences
ax4 = axes[1, 1]
rank_diff = fan_means - judge_means
colors_diff = ['green' if d > 0 else 'orange' for d in rank_diff]
bars_diff = ax4.bar(occupations, rank_diff, color=colors_diff)
ax4.set_xlabel('Occupation')
ax4.set_ylabel('Rank Difference (Fan - Judge)')
ax4.set_title('Preference Difference\nPositive = Fans Prefer More')
ax4.set_xticklabels(occupations, rotation=45, ha='right')
ax4.axhline(y=0, color='black', linestyle='-', linewidth=1)
ax4.grid(True, alpha=0.3)

for bar in bars_diff:
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., 
            height + (0.1 if height > 0 else -0.4),
            f'{height:+.1f}', ha='center', 
            va='bottom' if height > 0 else 'top', fontsize=9)

plt.tight_layout()
plt.savefig('occupation_analysis.png', dpi=300, bbox_inches='tight')

# Create age charts if age data exists
if has_age_data:
    fig2, axes2 = plt.subplots(1, 2, figsize=(14, 6))
    fig2.suptitle("Age Performance Analysis", fontsize=16)
    
    age_groups = age_stats.index.tolist()
    judge_age = age_stats[("judge_rank", "mean")]
    fan_age = age_stats[("fan_rank", "mean")]
    
    # Age vs judge ranking
    ax5 = axes2[0]
    ax5.plot(age_groups, judge_age, marker='o', linewidth=2, label='Judge')
    ax5.set_xlabel('Age Group')
    ax5.set_ylabel('Average Judge Rank')
    ax5.set_title('Age vs Judge Performance')
    ax5.legend()
    ax5.grid(True, alpha=0.3)
    
    # Age vs fan ranking
    ax6 = axes2[1]
    ax6.plot(age_groups, fan_age, marker='s', linewidth=2, color='red', label='Fan')
    ax6.set_xlabel('Age Group')
    ax6.set_ylabel('Average Fan Rank')
    ax6.set_title('Age vs Fan Popularity')
    ax6.legend()
    ax6.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('age_analysis.png', dpi=300, bbox_inches='tight')

# =========================
# 8. Results summary
# =========================

print("\n" + "="*60)
print("Key Findings Summary")
print("="*60)

# Best occupations
best_judge = occupation_stats[("judge_rank", "mean")].idxmin()
best_fan = occupation_stats[("fan_rank", "mean")].idxmin()
print(f"1. Highest judge-rated occupation: {best_judge}")
print(f"2. Highest fan-voted occupation: {best_fan}")

# Preference differences
pref_diff = {}
for occ in occupations:
    judge_val = occupation_stats.loc[occ, ("judge_rank", "mean")]
    fan_val = occupation_stats.loc[occ, ("fan_rank", "mean")]
    pref_diff[occ] = fan_val - judge_val

judge_favored = min(pref_diff, key=pref_diff.get)
fan_favored = max(pref_diff, key=pref_diff.get)
print(f"3. Judge's most preferred occupation: {judge_favored} (difference: {pref_diff[judge_favored]:.1f})")
print(f"4. Fans' most preferred occupation: {fan_favored} (difference: {pref_diff[fan_favored]:.1f})")

if has_age_data:
    best_judge_age = age_stats[("judge_rank", "mean")].idxmin()
    best_fan_age = age_stats[("fan_rank", "mean")].idxmin()
    print(f"5. Highest judge-rated age group: {best_judge_age}")
    print(f"6. Highest fan-voted age group: {best_fan_age}")

print("\nDetailed occupation rankings:")
print("-"*40)
for occ in occupations:
    judge_val = occupation_stats.loc[occ, ("judge_rank", "mean")]
    fan_val = occupation_stats.loc[occ, ("fan_rank", "mean")]
    diff = fan_val - judge_val
    count = int(occupation_stats.loc[occ, ("judge_rank", "count")])
    pref = "Fans prefer more" if diff > 0 else "Judges prefer more"
    print(f"{occ:20s} | Judge: {judge_val:5.1f} | Fan: {fan_val:5.1f} | Diff: {diff:+.1f} | {pref} | Sample: {count}")

print("\n" + "="*60)
print("Analysis complete! Charts saved as occupation_analysis.png")
if has_age_data:
    print("Age charts saved as age_analysis.png")
print("="*60)

plt.show()

In [ ]:
import pandas as pd
import numpy as np
import unicodedata
import matplotlib.pyplot as plt

# Load raw data and predictions
raw = pd.read_csv("2026_MCM_Problem_C_Data.csv")
pred = pd.read_excel("merged_data_with_predictions.xlsx")

def clean_name(x):
    """Clean and normalize celebrity names"""
    x = str(x).lower()
    x = unicodedata.normalize("NFKD", x)
    return "".join(c for c in x if c.isalpha())

# Apply name cleaning to both datasets
raw["name_clean"] = raw["celebrity_name"].apply(clean_name)
pred["name_clean"] = pred["celebrity_name"].apply(clean_name)

def categorize(ind):
    """Categorize industries into standardized groups"""
    if pd.isna(ind): return "Other"
    ind = str(ind).lower()
    if "actor" in ind: return "Actor"
    if "athlete" in ind or "sport" in ind: return "Athlete"
    if "singer" in ind or "musician" in ind: return "Music"
    if "tv" in ind or "reality" in ind: return "TV"
    return "Other"

# Apply industry categorization
raw["occupation_category"] = raw["celebrity_industry"].apply(categorize)

# Find age column
age_col = [c for c in raw.columns if "age" in c.lower()][0]

# Prepare basic info dataframe
info = raw[["name_clean","occupation_category",age_col]].drop_duplicates()
info = info.rename(columns={age_col:"age"})

# Merge dataframes
df = pred.merge(info, on="name_clean", how="left")

# Identify judge and fan vote columns
judge_cols = [c for c in df.columns if "Judge" in c or "judge" in c]
fan_cols = [c for c in df.columns if "Predicted_Fan_Votes" in c]

# Calculate totals
df["judge_total"] = df[judge_cols].sum(axis=1)
df["fan_total"] = df[fan_cols].sum(axis=1)

# Prepare analysis dataset
analysis = df[["name_clean","occupation_category","age","judge_total","fan_total"]].dropna()

# Create industry mapping codes
industry_map = {k:i+1 for i,k in enumerate(analysis["occupation_category"].unique())}
analysis["industry_code"] = analysis["occupation_category"].map(industry_map)

# Clean age data
analysis["age"] = pd.to_numeric(analysis["age"], errors="coerce")
analysis = analysis.dropna()

def topsis(X, w):
    """TOPSIS method for multi-criteria decision making"""
    # Normalize the matrix
    R = X / np.sqrt((X**2).sum(axis=0))
    # Calculate weighted normalized matrix
    V = R * w
    # Determine ideal and worst solutions
    ideal = V.max(axis=0)
    worst = V.min(axis=0)
    # Calculate distances
    Dp = np.sqrt(((V - ideal)**2).sum(axis=1))
    Dn = np.sqrt(((V - worst)**2).sum(axis=1))
    # Calculate relative closeness
    return Dn / (Dp + Dn)

# Prepare feature matrix
X = analysis[["age","industry_code","judge_total","fan_total"]].values

# Weight configurations
# Judge-dominated weights
w_judge = np.array([0.15,0.15,0.55,0.15])

# Fan-dominated weights  
w_fan = np.array([0.10,0.20,0.15,0.55])

# Apply TOPSIS with different weight schemes
analysis["Judge_TOPSIS"] = topsis(X, w_judge)
analysis["Fan_TOPSIS"] = topsis(X, w_fan)

# Calculate rankings
analysis["Judge_rank"] = analysis["Judge_TOPSIS"].rank(ascending=False)
analysis["Fan_rank"] = analysis["Fan_TOPSIS"].rank(ascending=False)

# Calculate ranking differences
analysis["Rank_gap"] = analysis["Fan_rank"] - analysis["Judge_rank"]

# Analyze industry bias
industry_bias = analysis.groupby("occupation_category").agg({
    "Judge_rank":"mean",
    "Fan_rank":"mean",
    "name_clean":"count"
}).rename(columns={"name_clean":"n"})

industry_bias["Preference_Index"] = industry_bias["Fan_rank"] - industry_bias["Judge_rank"]

# Print results
print("\n====== Industry Preference Index ======\n")
print(industry_bias.sort_values("Preference_Index", ascending=False))

# Calculate bias strength metrics
bias_strength = np.mean(np.abs(analysis["Rank_gap"]))
print("\nOverall mechanism bias strength:", round(bias_strength,2))
print("Maximum individual deviation:", analysis["Rank_gap"].abs().max())

# Create visualization
plt.figure(figsize=(6,5))
plt.scatter(analysis["Judge_rank"], analysis["Fan_rank"], alpha=0.7)
plt.plot([1,len(analysis)], [1,len(analysis)], linestyle="--")
plt.xlabel("Judge-based Ranking")
plt.ylabel("Fan-based Ranking")
plt.title("Ranking Discrepancy Between Evaluation Mechanisms")
plt.grid(True)
plt.show()

4th problem

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load data (assuming files are in the same folder)
df_raw = pd.read_excel('merged_data_with_predictions.xlsx')

# Focus on a specific season (e.g., Season 32)
SEASON = 32
df_season = df_raw[df_raw['season'] == SEASON].copy()

# --------------------------------------------------
# Extract judge score columns (week*_judge*_score)
# --------------------------------------------------
judge_cols = [c for c in df_season.columns if 'week' in c and 'judge' in c]

# Create (contestant, week) → average judge score mapping
records = []

for _, row in df_season.iterrows():
    name = row['celebrity_name']
    for col in judge_cols:
        if not pd.isna(row[col]):
            # Parse week number
            week = int(col.split('_')[0].replace('week', ''))
            records.append({
                'celebrity_name': name,
                'week': week,
                'pJ_raw': row[col]
            })

df_judge_long = pd.DataFrame(records)

# Average multiple judges for the same week
df_judge_week = (
    df_judge_long
    .groupby(['celebrity_name', 'week'], as_index=False)
    .agg(pJ=('pJ_raw', 'mean'))
)

# Get fan voting data
fan_col = f'Season{SEASON}_Avg_Fan_Pct'
df_fan = df_season[['celebrity_name', fan_col]].rename(columns={fan_col: 'pF'})

# Merge datasets
df = df_judge_week.merge(df_fan, on='celebrity_name', how='left')

# Calculate contestant statistics (mean and standard deviation)
judge_stats = (
    df
    .groupby('celebrity_name')
    .agg(
        mu_J=('pJ', 'mean'),
        sigma_J=('pJ', 'std')
    )
    .reset_index()
)

judge_stats['rank_J'] = judge_stats['mu_J'].rank(ascending=False, method='min')

# Set weights and stability protection factor
wJ, wF = 0.5, 0.5
alpha = 0.2  # Increase alpha to enhance stability factor's impact on S_star
eps = 1e-6

# Merge judge statistics
df = df.merge(judge_stats, on='celebrity_name', how='left')

# Calculate stability protection factor B
df['B'] = alpha * df['mu_J'] / (df['sigma_J'] + eps)

# Calculate comprehensive score S_star
df['S_star'] = (
    wJ * df['pJ'] +
    wF * df['pF'] +
    df['B']
)

# Set elimination criteria: elimination score (e.g., contestants with S_star below threshold)
threshold = 2.5  # Assuming contestants with S_star < 2.5 are eliminated

# Mark eliminated contestants
df['eliminated'] = df['S_star'] < threshold

# Data for remaining contestants
df_remaining = df[~df['eliminated']]

# Calculate weekly elimination stability: only consider contestants still in competition
elimination = (
    df_remaining
    .groupby('week')
    .agg(S_star_mean=('S_star', 'mean'))
    .reset_index()
)

# Output elimination scores (by week)
print("Weekly Elimination Stability (Mean Score):")
print(elimination[['week', 'S_star_mean']])

# Final stage score adjustment
K = 3
lambda_penalty = 0.05  # Reduce lambda_penalty value to avoid over-penalization

# Get final week
final_week = df['week'].max()
df_final = df[df['week'] == final_week].copy()

# Calculate final score (for champion ranking adjustment)
df_final['final_score'] = np.where(
    df_final['rank_J'] <= K,
    df_final['S_star'],
    df_final['S_star'] - lambda_penalty * (df_final['rank_J'] - K)
)

# Compare original system vs new system scores
df_final['S_original'] = wJ * df_final['pJ'] + wF * df_final['pF']  # Original score
df_final['S_A'] = df_final['S_star']  # System A
df_final['S_B'] = df_final['final_score']  # System B

# Print final scores and rankings to confirm differences
print("Final Scores and Rank J:")
print(df_final[['celebrity_name', 'rank_J', 'S_star', 'final_score']])

# --------------------------------------------------------
# 1️⃣ Compare champion judge rankings
champ = df_final.iloc[0]

# Create champion judge ranking comparison chart
plt.bar(
    ['Original', 'System A', 'System B'],
    [champ['rank_J']] * 3
)
plt.ylabel('Judge Rank of Champion')
plt.title('Champion Professional Ranking Comparison')
plt.show()

# --------------------------------------------------------
# 2️⃣ Original system vs new system score distribution
plt.hist(df_final['S_original'], bins=10, alpha=0.6, label='Original')
plt.hist(df_final['S_B'], bins=10, alpha=0.6, label='New System')
plt.legend()
plt.title('Final Score Distribution Comparison')
plt.show()

# --------------------------------------------------------
# 3️⃣ Single-week elimination stability comparison
plt.plot(elimination['week'], elimination['S_star_mean'], marker='o')
plt.xlabel('Week')
plt.ylabel('Mean Eliminated Score')
plt.title('Weekly Elimination Stability')
plt.show()